
# Impairment Detection Agent

This notebook shows, step‑by‑step, how the Strands agent ingests XML feeds, identifies impairments,
pulls scoring‑factor names from the Knowledge‑Base markdown, and emits the JSON payload
expected by Work‑stream 4.


## Installation and Dependencies

**What:** Install the core libraries needed for the detection agent.

**Why:** 
- `boto3` - Provides AWS SDK access to Bedrock LLMs and knowledge bases
- `strands-agents` - The agent framework that orchestrates tool calls and LLM interactions
- `numpy` - Used for vector similarity calculations when using local knowledge base
- `lxml` - Parses XML data feeds (application, RX, labs, MIB)

These dependencies allow the agent to ingest structured insurance data, query underwriting guidelines, and reason about medical impairments.


In [54]:
%pip install boto3 strands-agents numpy lxml

Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Configuration

**What:** Set up the model, knowledge base, and data source paths.

**Why:**
- **Flexible Knowledge Base**: Toggle between local markdown files (for development/demos) and AWS Bedrock Knowledge Base (for production). Set `kb_id = None` to use local files.
- **Model Selection**: Use Claude 3.7 Sonnet for its advanced reasoning capabilities needed to detect subtle medical patterns across multiple data sources.
- **Embedding Model**: Amazon Titan embeddings power semantic search across underwriting guidelines when using local knowledge base.
- **Mock Data Switching**: Easily switch between test cases (diabetes_cardiovascular vs hypertension) to validate agent behavior across different clinical scenarios.

This configuration makes the notebook portable - it can run locally with markdown files or connect to cloud infrastructure without code changes.


In [55]:

import os, json, boto3
import numpy as np
from collections import defaultdict
from lxml import etree
from strands import Agent, tool

# ---- Set these before running locally ----
# Knowledge base configuration - uncomment the line below to use Bedrock Knowledge Base instead of local files
#kb_id = 'YSWIGPQHRJ'
kb_id = None  # Reset to None to ensure clean state

model_id = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
embedding_model_id = 'amazon.titan-embed-text-v2:0'

# Mock data configuration
# mock_data_path = "../mock_data/hypertension"
mock_data_path = "../mock_data/diabetes_cardiovascular"

# Local knowledge base path
local_kb_path = "../underwriting_manual"


## Local Knowledge Base Setup

**What:** Create an in-memory vector database from markdown underwriting guidelines.

**Why:**
- **Development Independence**: Run the agent without AWS dependencies - perfect for demos, testing, or air-gapped environments.
- **Semantic Search**: Convert each markdown file into an embedding vector, allowing the agent to find the most relevant underwriting guidelines (e.g., searching "high blood pressure" matches the hypertension.md file).
- **Cost Efficiency**: Avoid Bedrock Knowledge Base charges during development and testing.
- **Transparency**: See exactly which documents the agent is using - helpful for debugging and understanding agent behavior.

The `cosine_similarity` function measures how closely a search query matches each document's semantic meaning, not just keyword matching. This mirrors how a human underwriter would recognize that "elevated A1C" relates to diabetes guidelines.


In [56]:
# Local Knowledge Base Setup
local_kb_store = None
bedrock_runtime = boto3.client('bedrock-runtime')

def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def create_embedding(text):
    """Create embedding using Amazon Titan model"""
    response = bedrock_runtime.invoke_model(
        modelId=embedding_model_id,
        body=json.dumps({"inputText": text})
    )
    embedding = json.loads(response['body'].read())['embedding']
    return np.array(embedding)

def load_local_knowledge_base():
    """Load markdown files from local underwriting manual and create embeddings"""
    global local_kb_store
    
    if 'kb_id' in globals() and kb_id is not None:
        print("Bedrock KB configured, skipping local KB loading...")
        return
    
    print(f"Loading local knowledge base from {local_kb_path}...")
    
    kb_documents = []
    
    # Find all markdown files in the underwriting manual directory
    if not os.path.exists(local_kb_path):
        print(f"Warning: Local KB path {local_kb_path} does not exist")
        return
    
    for filename in os.listdir(local_kb_path):
        if filename.lower().endswith('.md'):
            file_path = os.path.join(local_kb_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                print(f"✓ Loading {filename} ({len(content)} chars)")
                
                # Create embedding for the document
                embedding = create_embedding(content)
                
                kb_documents.append({
                    'filename': filename,
                    'content': content,
                    'embedding': embedding
                })
                
            except Exception as e:
                print(f"✗ Error loading {filename}: {e}")
    
    local_kb_store = kb_documents
    print(f"Local knowledge base loaded with {len(kb_documents)} documents")

# Load the local knowledge base if kb_id is not defined or None
if 'kb_id' not in globals() or kb_id is None:
    load_local_knowledge_base()
else:
    print("Using Bedrock Knowledge Base")


Loading local knowledge base from ../underwriting_manual...
✓ Loading hypertension.md (7391 chars)
✓ Loading type1_diabetes.md (9446 chars)
✓ Loading type2_diabetes.md (9187 chars)
✓ Loading lab_values.md (18969 chars)
Local knowledge base loaded with 4 documents


## Agent Tools - Part 1: Scratch Pad

**What:** An optional tool that sets up a temporary storage tool that will allow the agent maintain working memory across multiple reasoning steps.

**Why:**
The impairment detection process is complex and multi-step:
1. Scan all data sources to identify potential impairments
2. For each impairment, search the knowledge base
3. Extract scoring factors from guidelines
4. Cross-reference data feeds to find values for those factors
5. Compile evidence from multiple sources

**Without a scratch pad**, the agent would have to hold all this information in its context window and might lose track of what it's already discovered. The scratch pad lets it:
- Store a running list of impairments found
- Track which impairments have been fully analyzed
- Build up scoring factors and evidence incrementally

This is analogous to how a human underwriter uses notes while reviewing a case file.

*Note: This tool may or may not be needed depending on the context length of the model you are using and how long the agent needs to run.*


In [57]:
# Corrected tools that follow Strands documentation patterns

@tool
def scratch_fixed(action: str, key: str, value=None, agent=None):
    """Tool for temporary storage during agent execution - uses agent.state properly"""
    # Use agent state for persistence across tool calls
    scratch_data = agent.state.get('scratch_pad') or {}
    
    if action == 'append':
        if key not in scratch_data:
            scratch_data[key] = []
        scratch_data[key].append(value)
    elif action == 'set':
        scratch_data[key] = value
    elif action == 'get':
        return scratch_data.get(key)
    
    # Save back to agent state
    agent.state.set('scratch_pad', scratch_data)
    return 'ok'


## Agent Tools - Part 2: Knowledge Base Search

**What:** A tool that retrieves underwriting guidelines for specific medical conditions.

**Why:**
The agent needs access to authoritative underwriting rules to:
- **Identify Scoring Factors**: Each impairment has specific factors that affect risk (e.g., for diabetes: A1C level, duration, complications)
- **Understand Severity Thresholds**: Guidelines define what values are considered mild vs. severe (e.g., A1C < 7% vs. > 9%)
- **Know What Evidence to Look For**: The guidelines tell the agent which lab values, medications, or clinical findings are relevant

**Dual-Mode Design**: This function automatically switches between:
- **Local Mode**: Searches markdown files using vector similarity (useful for development)
- **Bedrock Mode**: Queries AWS Bedrock Knowledge Base (production-ready, managed service)

Think of this as the agent consulting the underwriting manual - just like a human underwriter would reference rating tables and guidelines when evaluating a condition.


In [58]:

kb_rt = boto3.client('bedrock-agent-runtime')

@tool
def kb_search(canonical_term: str):
    """Return markdown for the top KB hit from either local or Bedrock knowledge base."""
    
    if ('kb_id' not in globals() or kb_id is None) and local_kb_store:
        # Use local knowledge base
        print(f"Searching local KB for: {canonical_term}")
        
        # Create embedding for the search query
        query_embedding = create_embedding(canonical_term)
        
        # Find the most similar document
        best_match = None
        best_similarity = -1
        
        for doc in local_kb_store:
            similarity = cosine_similarity(query_embedding, doc['embedding'])
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = doc
        
        if best_match:
            print(f"Best match: {best_match['filename']} (similarity: {best_similarity:.3f})")
            return best_match['content']
        else:
            return "No matching documents found in local knowledge base."
    
    else:
        # Use Bedrock Knowledge Base
        print(f"Searching Bedrock KB for: {canonical_term}")
        resp = kb_rt.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': canonical_term},
            retrievalConfiguration={'vectorSearchConfiguration': {'numberOfResults': 1}}
        )
        print(resp)
        # According to official AWS documentation, the field is 'text', not 'text_markdown'
        return resp['retrievalResults'][0]['content']['text']


## Agent System Prompt

**What:** Instructions that define the agent's role, workflow, and output format.

**Why:**
This prompt orchestrates a sophisticated multi-step reasoning process that mirrors how experienced underwriters work:

**Step 1: Initial Scan** - "What conditions does this applicant have?"
- Review application questionnaire answers
- Check prescription history for medications
- Analyze lab results for abnormal values
- Review MIB records for prior insurance applications

**Step 2: Per-Impairment Deep Dive** - "How do I score this condition?"
- Query the knowledge base for relevant guidelines
- Extract the specific scoring factors needed (e.g., A1C, duration, complications)

**Step 3: Evidence Gathering** - "What data supports this assessment?"
- Cross-reference all data sources
- Find values for each scoring factor
- Document where each piece of evidence came from

**Step 4: Structured Output** - "Package findings for the scoring agent"
- Format results as JSON with impairment ID, scoring factors, and evidence
- This payload becomes input for Workstream 4 (risk scoring)

The prompt emphasizes **deduplication** because the same condition often appears in multiple sources (e.g., diabetes mentioned in application, RX history, labs, and MIB). The agent must consolidate these into a single coherent assessment.


In [59]:

PROMPT = """You are a senior life insurance underwriter. Your job is to analyze the data stream for an application and identify impairments, 
scoring factors (based on the knowledge base), and evidences for those impairments. 
1. Scan the XML feeds (application, Rx, labs, MIB) for impairment evidence and write out an initial list of impairments.
Then for each impairment in your scratch pad, do the following:
2. Call kb_search() once and treat the markdown returned as authoritative.
3. Use the ratings tables in the returned markdown to determine a list of "scoring factors" are required to completely score that impairment and write them out. 
4. Search through the XML feeds to consolidate the values for each scoring factor, and the list of evidence for that impairment. 
5. Write out the scoring factors and evidence for that impairment.

Repeat this process for each impairment you find. Deduplicate any impairment that is found in multiple XML feeds into one listng. 

Once you have completed this process for all impairments, return the following JSON:
```json   
   [
     {
       "impairment_id": "diabetes",
       "scoring_factors": {"A1C": 8.2, "Neuropathy": true},
       "evidence": ["Rx: insulin …", "Lab: A1C 8.2 %"]
     }
   ]
```
"""


## Test Utility (Optional)

**What:** A simple function to verify the local knowledge base is working correctly.

**Why:**
Before running the full agent, it's useful to validate:
- All markdown files loaded successfully
- Embeddings were created properly
- Semantic search returns relevant documents

Uncomment the last line (`test_local_kb()`) to see:
- Which documents are in the knowledge base
- What gets returned when searching for "diabetes"
- Whether the similarity scoring is working

This is particularly helpful when troubleshooting or adding new underwriting guidelines to the knowledge base.


In [60]:
# Test local knowledge base functionality
def test_local_kb():
    """Test the local knowledge base search functionality"""
    if ('kb_id' not in globals() or kb_id is None) and local_kb_store:
        print(f"Local KB contains {len(local_kb_store)} documents:")
        for doc in local_kb_store:
            print(f"  - {doc['filename']}")
        
        # Test a search
        print("\nTesting search for 'diabetes':")
        result = kb_search("diabetes")
        print(f"Result length: {len(result)} characters")
        print("First 200 characters:", result[:200] + "..." if len(result) > 200 else result)
    else:
        print("Local KB not available or Bedrock KB is configured")

# Uncomment to test the local knowledge base
# test_local_kb()


## Load Mock Application Data

**What:** Read synthetic JSON files that simulate real insurance data feeds.

**Why:**
In production, an underwriter receives data from multiple sources:
- **Application JSON** - Applicant demographics, medical history questionnaire, coverage details
- **Prescription History (RX)** - Medication fills from pharmacy databases (IntelliScript)
- **Lab Results** - Blood work, urinalysis, diagnostic tests
- **MIB Response** - Medical Information Bureau records from prior insurance applications
- **RX CSV** - Alternative prescription data format (sometimes CSV, sometimes XML)

**Why JSON instead of XML?** The mock data has been converted to JSON with:
- Masked company names and addresses (to anonymize the source)
- Preserved clinical values (labs, medications, dosages, indications)
- Randomized structure (to prevent exact schema matching)

This lets us test with realistic data scenarios:
- **diabetes_cardiovascular/** - Complex case with Type 2 Diabetes, hypertension, and cardiovascular risk factors
- **hypertension/** - Simpler case with controlled blood pressure and excellent labs

The agent must handle both scenarios and correctly extract impairments regardless of data structure.


In [61]:
# Load mock data from ../mock_data directory
def load_mock_data():
    import os
    
    
    mock_data = {}
    
    # Load all XML files from specified directory
    # Load all XML files from the specified directory
    files = {}
    for filename in os.listdir(mock_data_path):
        if filename.lower().endswith('.json'):
            key = filename.replace('.json', '')
            files[key] = filename
    
    for key, filename in files.items():
        file_path = os.path.join(mock_data_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                mock_data[key] = f.read()
            print(f"✓ Loaded {filename}")
        except FileNotFoundError:
            print(f"✗ Could not find {filename}")
            mock_data[key] = ''
    
    return mock_data

# Load the mock data
mock_data = load_mock_data()
print(f"\nLoaded {len([v for v in mock_data.values() if v])} files successfully")


✓ Loaded mock_lab_results.json
✓ Loaded mock_rx_data.json
✓ Loaded mock_intelliscript_rx.json
✓ Loaded mock_application.json
✓ Loaded mock_mib_response.json

Loaded 5 files successfully


## Initialize the Detection Agent

**What:** Create an Agent instance with the system prompt and knowledge base search tool.

**Why:**
The Strands Agent framework connects three critical components:

1. **System Prompt** - Defines the agent's reasoning workflow and output format
2. **Tools** - Gives the agent the ability to search knowledge bases (the `kb_search` function)
3. **Model** - The LLM (Claude 3.7 Sonnet) that performs the reasoning

When the agent runs, it will:
- Read the system prompt to understand its task
- Analyze the data feeds provided
- Decide when to call `kb_search()` to retrieve underwriting guidelines
- Reason about the results and format the final JSON output

This is the "brain" of the detection system - the orchestrator that decides what to do and when.


In [62]:
# Updated detector with corrected tools and message handling
detector = Agent(
    system_prompt=PROMPT,
    tools=[kb_search],
    model=model_id,
)


## Run Detection Function

**What:** A utility function that packages data feeds and calls the agent.

**Why:**
This function provides a clean interface to:
- **Use mock data by default** - Makes testing easy without manually loading files
- **Support custom data** - Can override with specific JSON strings if needed
- **Extract JSON output** - Parses the agent's response and handles markdown code blocks

**The workflow:**
1. Gather all data feeds (application, RX, labs, MIB) into a single message
2. Send to the agent for analysis
3. Agent performs multi-step reasoning (scan → search KB → extract factors → gather evidence)
4. Parse the returned JSON (handles cases where agent wraps output in ```json blocks)
5. Return structured data ready for Workstream 4 (scoring agent)

This abstraction makes it easy to integrate the detection agent into a larger workflow or API endpoint.


In [63]:

    

def run_detection(application='', rx='', labs='', mib='', use_mock_data=True):
    """Utility to run the agent in‑notebook"""
    
    # Use mock data by default if no specific JSON is provided
    if use_mock_data and not any([application, rx, labs, mib]):
        feeds = mock_data.copy()
        print("Using mock data from ../mock_data directory")
    else:
        feeds = {
            'application': application,
            'rx': rx,
            'labs': labs,
            'mib': mib,
        }
    
    # Create a simple string message with all data feeds
    message = f"Here are the data feeds to analyze for impairments:\n\n{feeds}"
    
    # Call the agent with a simple string message (correct way according to Strands docs)
    res = detector(message)
    print("Agent response:")
    print(res)
    import re

    # Extract JSON from between ```json ... ``` tags if present
    res_str = str(res)
    json_match = re.search(r"```json\s*(.*?)\s*```", res_str, re.DOTALL)
    if json_match:
        res_str = json_match.group(1)
    else:
        # If no markdown code block, try to find JSON array directly
        json_match = re.search(r"\[.*\]", res_str, re.DOTALL)
        if json_match:
            res_str = json_match.group(0)
    
    return json.loads(res_str)

# Now you can run detection with mock data easily:
# sample_output = run_detection()  # Uses mock data automatically
# print(json.dumps(sample_output, indent=2))


## Execute Detection on Mock Data

**What:** Run the complete impairment detection workflow on the selected test case.

**Why:**
This is where everything comes together:

**Input:** Multiple JSON data feeds from various sources (application, RX history, labs, MIB)

**Process:** The agent will:
1. Scan all feeds to identify potential impairments
2. For each impairment (e.g., diabetes, hypertension):
   - Search the knowledge base for relevant guidelines
   - Determine what scoring factors are needed
   - Find values for those factors across all data sources
   - Compile evidence supporting the diagnosis

**Output:** Structured JSON listing each impairment with:
- `impairment_id` - Canonical name (e.g., "type2_diabetes")
- `scoring_factors` - Key-value pairs needed for risk scoring (e.g., {"HbA1c": 7.2, "duration_years": 4})
- `evidence` - List of supporting data points with sources

**Why this matters:** This output becomes the input to Workstream 4 (scoring agent), which will:
- Look up rating tables for each impairment
- Apply the scoring factors to calculate debits/credits
- Generate a final risk score

The detection agent's job is to **find and structure the relevant medical information** so the scoring agent can focus on **applying underwriting rules and calculating risk**.


In [64]:
# Run the impairment detection using mock data
print("=== Running Impairment Detection with Mock Data ===")
print(f"Knowledge Base Mode: {'Local' if 'kb_id' not in globals() or kb_id is None else 'Bedrock'}")
print()

try:
    # This will automatically use the mock data from ../mock_data
    results = run_detection()
    
    print("\n=== Detection Results ===")
    print(json.dumps(results, indent=2))
    
except Exception as e:
    print(f"\n\nError running detection: {e}")
    print("\nMake sure:")
    print("\n1. Your AWS credentials are configured")
    if 'kb_id' not in globals() or kb_id is None:
        print("\n2. The local underwriting manual files are accessible")
        print("\n3. You have access to the Bedrock embedding model")
    else:
        print("\n2. The KB_ID is set correctly")
        print("\n3. You have access to the specified Bedrock model and knowledge base")



=== Running Impairment Detection with Mock Data ===
Knowledge Base Mode: Local

Using mock data from ../mock_data directory
I'll analyze the data feeds to identify impairments, determine scoring factors, and gather evidence for each impairment. Let me start by scanning the XML feeds for impairment evidence.

Based on my initial scan of the XML feeds, I've identified these potential impairments:
1. Type 2 Diabetes
2. Hypertension
3. Hyperlipidemia (High Cholesterol)

Let me analyze each impairment in detail.

## 1. Type 2 Diabetes
Tool #1: kb_search
Searching local KB for: diabetes
Best match: type2_diabetes.md (similarity: 0.401)
Based on the knowledge base for diabetes, I need to identify the following scoring factors:
- HbA1c
- Duration of diabetes
- Treatment regimen
- Complications (retinopathy, nephropathy, neuropathy, etc.)
- Comorbidities 

Now I'll search the XML feeds to gather values for these factors and evidence.

### Diabetes Scoring Factors:
- **HbA1c**: 7.2% (from lab re

## Summary: What Just Happened?

The detection agent just performed sophisticated medical record analysis that would normally require:
- **A trained underwriter** to review multiple data sources
- **30-60 minutes** to manually cross-reference medications, labs, and medical history
- **Deep knowledge** of underwriting guidelines and rating factors

**What the agent did automatically:**
1. ✅ Identified all medical impairments across 5 different data sources
2. ✅ Consulted underwriting guidelines to determine scoring requirements
3. ✅ Cross-referenced data to find specific values (A1C, blood pressure, medication adherence, etc.)
4. ✅ Compiled evidence trails showing where each data point came from
5. ✅ Formatted structured output ready for automated risk scoring

**Key advantages of the agentic approach:**
- **Consistency**: Same analysis every time, no human variability
- **Speed**: Processes cases in seconds instead of minutes
- **Completeness**: Never misses relevant data scattered across sources
- **Auditability**: Documents evidence chain for compliance and quality review
- **Scalability**: Can process thousands of cases without additional headcount

**Next step:** Take this output to `scoring_agent.ipynb` to calculate the final risk score using rating tables and underwriting rules.
